In [1]:
import python_vali as vali
import numpy as np
import av

In [2]:
input = "../tests/data/test.mp4"
output = 'output.mp4'

target_w = 640
target_h = 360

In [3]:
# GPU-accelerated decoder
pyDec = vali.PyDecoder(
    input,
    {},
    gpu_id=0)

# GPU-accelerated resizer
pyRes = vali.PySurfaceResizer(pyDec.Format, gpu_id=0)

# GPU-accelerated encoder
nvEnc = vali.PyNvEncoder(
    {
        "preset": "P4",
        "codec": "h264",
        "s": str(target_w) + 'x' + str(target_h),
        "bitrate": "1M",
        "fps": f"{pyDec.Framerate}",
    },
    gpu_id=0
)

# Muxer
dstFile = av.open(output, 'w')
out_stream = dstFile.add_stream('h264', rate=int(pyDec.Framerate))
out_stream.width = target_w
out_stream.height = target_h

In [4]:
def make_packet(
        video_packet: np.ndarray,
        pkt_data: vali.PacketData,
        out_stream: av.VideoStream) -> av.Packet:

    pkt = av.packet.Packet(bytearray(video_packet))
    pkt.stream = out_stream
    pkt.pts = pkt_data.pts
    pkt.dts = pkt_data.dts

    return pkt

In [ ]:
# Decoded Surface
surf_src = vali.Surface.Make(
    format=pyDec.Format,
    width=pyDec.Width,
    height=pyDec.Height,
    gpu_id=0)

# Resized Surface
surf_dst = vali.Surface.Make(
    format=pyDec.Format,
    width=target_w,
    height=target_h,
    gpu_id=0
)

# Numpy array which contains encoded packet
packet = np.ndarray(
    dtype=np.uint8,
    shape=())

pkt_data = vali.PacketData()

while True:
    # Decode single Surface
    success, details = pyDec.DecodeSingleSurface(surf_src, pkt_data)
    if not success:
        print(details)
        break

    # Resize
    success, details = pyRes.Run(surf_src, surf_dst)
    if not success:
        print(details)
        break

    # Encode. If there's a compressed packet, write to file
    got_pkt = nvEnc.EncodeSingleSurface(surf_dst, packet)
    if got_pkt:
        av_pkt = make_packet(packet, pkt_data, out_stream)
        dstFile.mux(av_pkt)

# Encoder is async, so we need to flush it in the end
while True:
    got_pkt = nvEnc.FlushSinglePacket(packet)
    if got_pkt:
        av_pkt = make_packet(packet, pkt_data, out_stream)
        dstFile.mux(av_pkt)
    else:
        break

# Finish pending operations on file and close it
dstFile.close()

TaskExecInfo.END_OF_STREAM
